In [ ]:
# Script for kelp particle simulation

In [ ]:
%matplotlib inline
from parcels import FieldSet, Field, VectorField, ParticleFile, ParticleSet, JITParticle, AdvectionRK4, plotTrajectoriesFile, Variable
from parcels import ErrorCode
from parcels import rng as random
import numpy as np
import math
from datetime import timedelta, datetime
from operator import attrgetter

In [ ]:
class SampleParticle(JITParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.

In [ ]:
filenames = {'U': "GlobCurrent_example_data/20*.nc",
             'V': "GlobCurrent_example_data/20*.nc"}
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)
pset = ParticleSet.from_list(fieldset=fieldset, 
                            pclass=SampleParticle,
                            lon=(17),     
                            lat=(-33),
                            repeatdt=3600)      

In [ ]:
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()

In [ ]:
def SampleDistance(particle, fieldset, time): # Function measuring distance
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat

In [ ]:
def SampleAge(particle, fieldset, time):
    particle.age = time

In [ ]:
# Drag Approach 1
def KelpDrag(particle, fieldset, time):
        (u, v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        kelpvel = ((u*2)*0.5*0.8)/1
        particle.lon += kelpvel * particle.dt
        particle.lat += kelpvel * particle.dt
k_drag = pset.Kernel(KelpDrag)  # casting the KelpDrag function to a kernel object

In [ ]:
# Drag Approach 2
def KelpDrag(particle, fieldset, time):
        (u1, v1) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)

        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1]
        lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)

        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2]
        lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)

        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3]
        kelpvel = ((((u1 + 2*u2 + 2*u3 + u4) / 6.)*0.03))/1
        particle.lon += kelpvel * particle.dt
        particle.lat += kelpvel * particle.dt 
        
k_drag = pset.Kernel(KelpDrag)  # casting the KelpDrag function to a kernel object

In [ ]:
# Execution of particles with custom kernel/s, ie. kelp particle simulation

output_file = pset.ParticleFile(name="KelpFloat.nc", outputdt=timedelta(days=100)) # the file name and the time step of the outputs


pset.execute(AdvectionRK4 + k_drag, 
             runtime=timedelta(days=100),
             dt=timedelta(minutes=5),
             output_file=output_file,
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
pset.show('KelpFloat.nc', domain={'N':-31, 'S':-35, 'E':20, 'W':15}, show_time=datetime(2002, 4, 11, 0))

In [ ]:
pset.show(field = 'vector', savefile='KelpFloat_A', show_time=datetime(2002, 7, 20, 0))
pset.show(field = 'vector', savefile='KelpFloat_B', domain={'N':-31, 'S':-35, 'E':20, 'W':15}, show_time=datetime(2002, 7, 20, 0))

In [ ]:
plotTrajectoriesFile('KelpFloat.nc', mode='hist2d', bins=[10, 10]); #Histogram of number of particle observations per bin

In [ ]:
plotTrajectoriesFile('KelpFloat.nc', mode = 'movie2d_notebook')